In [1]:
import numpy as np

# Hello once more! By now you should have worked through QAS_Basics and QAS_Clouds. Hopefully, you are feeling a little more comfortable with understanding what this project is about.
# Now, we take the project in its current (somewhat) simplified state and transform it into something more physical and exciting!

# Specifically, we are going to add parameters to sample for. Doppler b and Velocity will remain unchanged but we can sample now for total (3D) density, instead of column density. We will still need to know the column density for voigt fitting so, we will also need the *length* of our line of sight. 
# This is because N = 3D Density (rho) * length of our line of sight (los)

# But there are some other factors that determine how much of the available column density is split into the different species. So far we have only sampled with Si-II (1526), but it is necessary to sample over all available absorptions for all available species. 
# Lets imagine we have a cloud that has both Silicon and Magnesium present in gas form. Not only will they be present, but some (if not all) of their ionization states will be present too. 

# Now the cloud has **1** value for its column density, but the Silicon and Magnesium do not necessarily split this value. Instead the cloud's ***metallicity*** will determine how much of that column density will be Silicon and how much will be Magnesium 

# Now that the column density is distributed between the two different elements, it will be split again amongst the different ionization states present. The amount that it will be split by is determined by the ***ionization fraction***. This fraction is determined by the cloud's ***3D density (rho)*** and its ***temperature***

# So now our parameters will be ***3D Density (rho), Length of line of sight (los), Temperature, Metallicity, Doppler b, and Velocity*** bringing our total to 6 free parameters per cloud 

In [11]:
# Here is an example of how this looks mathematically

total_rho    = 10**(1.2)   #total density of cloud
Metallicity  = 0.7           #total metallicity of cloud
los          = 10**(21.2)    #length of the line of sight
T            = 10**(3.6)     #Temperature of cloud
b            = 12.3          #Doppler b of cloud
v            = 100.          #Velocity of cloud

In [12]:
#Lets assume this cloud is made of Mg I,II and Si I,II

f_solar_Mg = 10. ** (-9.22)
f_solar_Si = 3.24 * 10.**(-4.48)   #scaled solar abundances of Si, Mg 

In [13]:
total_N = total_rho * los

In [14]:
N_Si = total_N * Metallicity * f_solar_Si  #total SI column Density
N_Mg = total_N * Metallicity * f_solar_Mg  #total Mg column Density

In [15]:
print ("The total logN for Si is",np.log10(N_Si),"and the total logN for Mg is",np.log10(N_Mg))

The total logN for Si is 18.275643050220868 and the total logN for Mg is 13.025098040014255


# Now we need to determine how the total logN for Si is split up between Si I and II and likewise for Mg. 
# The Ionization fraction is determined by the Saha ionization equation. It is big and ugly so lets lean off someone else's work!
# Luckily there is a program called, CLOUDY, which will give us the ionization fraction if we input the necessary parameters.
# Even more lucky, S. Bird has kindly made available the table that CLOUDY produces for a given rho and temp. lets load it now

In [29]:
def frac_from_dens(redshift,density,temp,species,ion):  #useful code to produce ionization fraction
    
    cloudy = np.load('cloudy_table.npz')   #load table

    table = cloudy['table']
    #[redshift,density,temperature,species,ion]#
    
    rrange = np.arange(7,-1,-1) #redshift range from S Bird Cloudy tables
    trange = np.arange(3.,8.6,0.05) #temp range from S Bird Cloudy tables
    drange = np.arange(-7.,4.,0.2) #drange range from S Bird Cloudy tables
    
    spec = np.array(["H", "He", "C", "N", "O", "Ne", "Mg", "Si", "Fe"])
    
    trange = trange.round(3) #formatting necessary to read table

    r = np.where(rrange == redshift)
    t = np.where(trange == temp)
    el = np.where(spec == species)
    
    r = int(r[0])
    t = int(t[0])
    el = int(el[0])
    
    ion = ion-1 #account for formatting of np arrays
    
    frac = []
    for i in range (len(drange)):
        frac.append(table[r,i,t,el,ion])
    frac = np.array(frac)
    frac = 10**(frac)
    
    frac_dens = np.interp(density,drange,frac)
    
    return frac_dens

In [39]:
# First lets walk through the Silicon, all inputs must be in array form!
redshift = np.array([0.0])  #necessary input parameter for CLOUDY 
density = np.array([np.log10(total_rho)]) #need log vals of Density and temp
temp = np.array([np.log10(T)])
species = np.array(['Si','Si','Mg','Mg'])
ion = np.array([1,2,1,2])   #Accounts for Si I,II and MgI,II

In [40]:
print(frac_from_dens(redshift,density,temp,species[0],ion[0])) #fraction of Si that is Si-I

[6.44169266e-05]


In [41]:
print(frac_from_dens(redshift,density,temp,species[1],ion[1])) #fraction for Si II

[1.]


In [42]:
print(frac_from_dens(redshift,density,temp,species[2],ion[2])) #fraction for Mg I

[0.00063826]


In [43]:
print(frac_from_dens(redshift,density,temp,species[3],ion[3])) #fraction for Mg II

[1.]


In [45]:
# Now lets find our indvidual column densities for all 4 of our species

f_Si_I = (frac_from_dens(redshift,density,temp,species[0],ion[0]))
f_Si_II = (frac_from_dens(redshift,density,temp,species[1],ion[1]))
f_Mg_I = (frac_from_dens(redshift,density,temp,species[2],ion[2]))
f_Mg_II = (frac_from_dens(redshift,density,temp,species[3],ion[3]))

In [47]:
N_Si_I = N_Si * f_Si_I   
N_Si_II = N_Si * f_Si_II

logN_Si_I = np.log10(N_Si_I)
logN_Si_II = np.log10(N_Si_II)

In [48]:
N_Mg_I = N_Mg * f_Mg_I
N_Mg_II = N_Mg * f_Mg_II

logN_Mg_I = np.log10(N_Mg_I)
logN_Mg_II = np.log10(N_Mg_II)

In [49]:
print("The log Column Density of Si-I is",logN_Si_I,"and Si-II is ",logN_Si_II)

The log Column Density of Si-I is [14.08464305] and Si-II is  [18.27564305]


In [50]:
print("The log Column Density of Mg-I is",logN_Mg_I,"and Mg-II is ",logN_Mg_II)

The log Column Density of Mg-I is [9.83009804] and Mg-II is  [13.02509804]


# While this set up may seem more complicated, there is actually something really cool going on here. We started with 6 parameters, yet we now have the capability to produce 4 distint absorbers (Si-I/II and Mg-(I/II))

# Of course, this will alter our set up, as now we need to account for the extra parameters and calculations - on to the next notebook!!